# Assignment 4AB. Spatial Interpolation
## NCSU GIS/MEA582: Geospatial Modeling and Anaylsis

Spring 2025

[Assignment 4AB: Spatial Interpolation]
(https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/topics/spatial_interpolation.html)

Authors: Corey White & Helena Mitasova

## Install GRASS in Colab

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev
print("INSTALLATION COMPLETE")

Check that the installation was successful by running the following cell.

In [ ]:
!grass --config version

Download the assignment data into the `/content` directory.

In [ ]:
!wget -c https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.zip -O nc.zip
!wget -c https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/deviations_color.txt -O deviations_color.txt
!wget -c https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/precip_color.txt -O precip_color.txt
!unzip nc.zip

Create a new mapset for the assigment called `assignment4ab` and set the region to `elevation`.

In [ ]:
!grass -c -e nc_spm_08_grass7/assignment4ab

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys
import matplotlib.pyplot as plt
from PIL import Image


# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init("nc_spm_08_grass7/assignment4ab")


## Spatial interpolation and approximation I: methods (4A)

### Compute Voronoi polygons

- Display the polygons with centroids.
- Find the column name where z is stored and convert the polygons to raster.
- Compute aspect to evaluate the surface geometry. 

In [ ]:
%%bash
g.region rural_1m -p
v.voronoi elev_lid792_randpts output=elev_vor

Display the voroni map

In [ ]:
# Create Map instance
voronoi_map = gj.Map()
# Add a vector to the map
voronoi_map.d_vect(map="elev_vor", size=1, flags="c", type="area,centroid")
# Display map
voronoi_map.show()

Find the column name where z is stored and convert the polygons to raster.

In [ ]:
!v.info -c elev_vor

Compute aspect to evaluate the surface geometry. 

In [ ]:
%%bash
v.to.rast elev_vor output=elev_vor_1m attrcolumn=value use=attr
r.colors elev_vor_1m color=elevation
r.slope.aspect elevation=elev_vor_1m aspect=asp_vor_1m

Display the resulting elevation map and aspect map. 

In [ ]:
# Create Map instance
elevation_map = gj.Map(filename="output/elev_voronoi.png")
# Add the elevation raster to the map
elevation_map.d_rast(map="elev_vor_1m")
# Add the aspect raster to the map
elevation_map.d_rast(map="asp_vor_1m")
elevation_map.d_vect(map="elev_lid792_randpts", size=1, flags="c", color="red")
# Display map
elevation_map.show()

Optionally, you can view it in 3D perspective (switch off all layers except for elev_vor_1m and switch to 3D view). 

In [ ]:
# Create Map instance
elevation_map3d = gj.Map3D(filename="output/elev_vor_1m_3D.png")
# Add the elevation raster to the map
elevation_map3d.render(elevation_map="elev_vor_1m", color_map="elev_vor_1m", perspective=20)
elevation_map3d.show()

### Interpolation using IDW

- Set region and resolution, find a column name where z is stored.
- Interpolate DEM using IDW, check the result using aspect. 

In [ ]:
%%bash
g.region rural_1m -p
v.info -c elev_lid792_randpts
v.surf.idw elev_lid792_randpts output=elev_idw_1m column=value
r.colors elev_idw_1m color=elevation
r.slope.aspect elevation=elev_idw_1m aspect=asp_idw_1m

Display the idw map

In [ ]:
# Create Map instance
idw_map = gj.Map(filename="output/elev_idw.png")
# Add the elevation raster to the map
idw_map.d_rast(map="elev_idw_1m")
# Add the aspect raster to the map
idw_map.d_rast(map="asp_idw_1m")
idw_map.d_vect(map="elev_lid792_randpts", size=2, flags="c", color="red")
# Display map
idw_map.show()

Design experiment that elucidates the impact of IDW parameters on the surface, focus on the impact of:


    - exponent e.g., power=0.5, 1, 5 (2 is the default)
    - number of neighboring points e.g., npoint=1, 5, 20, 60 (12 is the default)

Include selected images (e.g. hillshade or aspect) and relevant stats (e.g., mean, min, max from r.univar, histogram) that highlight the differences in the resulting surfaces into your report.

Check the surface interpolated with default parameters using 3D view.

> Do not forget to switch off everything except for the interpolated elevations and set fine resolution to 1.

You can use constant color for the surface to highlight its structure.

Save an image for your report. 

In [ ]:

def create_histogram(raster, filename):
    hist_img = gj.Map(filename=filename)
    hist_img.d_histogram(map=raster)
    hist_img.show()
    return filename

def create_map(elevation, aspect, filename):
    # Create Map instance
    idw_map = gj.Map(filename=filename)
    # Add the elevation raster to the map
    idw_map.d_rast(map=elevation)
    # Add the aspect raster to the map
    idw_map.d_rast(map=aspect)
    idw_map.d_vect(map="elev_lid792_randpts", size=2, flags="c", color="red")
    # Display map
    idw_map.show()
    return filename
    
def get_stats(elevation):
    univar = gs.parse_command("r.univar", map=elevation, flags="ge")
    min_val = float(univar["min"])
    max_val = float(univar["max"])
    mean_val = float(univar["max"])
    return {"min": min_val, "max": max_val, "mean": mean_val}
    

power=[0.5, 1, 2, 3, 5]
npoint=[1, 5, 20, 60]

def run_idw_experiment(power=[0.5, 1, 2, 3, 5], npoint=[1, 5, 20, 60]):
    gs.run_command("g.region", region="rural_1m", flags="p")
    tmp = []
    current_row = 0
    current_col = 0

    for p in power:
        for n in npoint:
            elev_output = f"elev_idw_1m_pow{p}_np{n}"
            aspect_output = f"asp_idw_1m_pow{p}_np{n}"
            image_output = f"output/elev_idw_pow{p}_np{n}.png"
            hist_output = f"output/hist_elev_idw_pow{p}_np{n}.png"
            gs.run_command("v.surf.idw", input="elev_lid792_randpts", output=elev_output, column="value", power=p, npoint=n)
            gs.run_command("r.colors", map=elev_output, color="elevation")
            gs.run_command("r.slope.aspect", elevation=elev_output, aspect=aspect_output)

            stats = get_stats(elev_output)
            create_map(elev_output, aspect_output, image_output)
            create_histogram(elev_output,hist_output)

            results = {
                "power": p,
                "npoint": n,
                "row": current_row,
                "column": current_col,
                "elevation": elev_output,
                "aspect": aspect_output,
                "image": image_output,
                "histogram": hist_output,
                "stats": stats
            }
            tmp.append(results)
            current_col = current_col + 1
            if current_col == columns:
                current_col = 0

        current_row = current_row + 1
        if current_row == rows:
            current_row = 0
        
    return tmp
        
        

In [ ]:
power=[0.5, 1, 2, 3, 5]
npoint=[1, 5, 20, 60]
runs = run_idw_experiment(power=power, npoint=npoint)

In [ ]:
power=[0.5, 1, 2, 3, 5]
npoint=[1, 5, 20, 60]
runs = run_idw_experiment(power=power, npoint=npoint)
fig = plt.figure(figsize=(20, 18))
columns = len(npoint)
rows = len(power)
print(f"Rows: {rows}, Columns: {columns}")
grs = fig.add_gridspec(nrows=rows, ncols=columns) # Row, Column

for run in runs:
    ax1 = fig.add_subplot(grs[run['row'], run['column']])
    ax1.axis('off')
    fig.subplots_adjust(hspace=0, wspace=0.5)
    img1 = Image.open(run["histogram"])
    imgplot = plt.imshow(img1)
    ax1.set_title(f"Power: {run['power']}, npoint: {run['npoint']}, mean: {run['stats']['mean']:.4f}",{"fontsize":12, "fontweight":"bold"})
    
    
plt.tight_layout()
plt.savefig("output/idw_hist_experiment.png",bbox_inches='tight', dpi=300)

In [ ]:
fig = plt.figure(figsize=(20, 18))
columns = len(npoint)
rows = len(power)
grs = fig.add_gridspec(nrows=rows, ncols=columns) # Row, Column
for run in runs:
    ax1 = fig.add_subplot(grs[run['row'], run['column']])
    ax1.axis('off')
    fig.subplots_adjust(hspace=0, wspace=0.5)
    img1 = Image.open(run["image"])
    imgplot = plt.imshow(img1)
    ax1.set_title(f"Power: {run['power']}, npoint: {run['npoint']}, mean: {run['stats']['mean']:.4f}",{"fontsize":12, "fontweight":"bold"})
       
plt.tight_layout()
plt.savefig("output/idw_elev_map_experiment.png",bbox_inches='tight', dpi=300)

### Compute DEM from contours

Compute DEM from contours using linear interpolation between isolines: 

In [ ]:
%%bash

g.region rural_1m -p
v.to.rast elev_lid792_cont1m output=el_lid792_cont1m attrcolumn=level use=attr
r.surf.contour el_lid792_cont1m output=el_rcont
r.colors el_rcont color=elevation

Check the result using a 2D aspect map or view el_rcont in 3D.
In 3D set view from SE and light from NW to reveal subtle geometry. 

In [ ]:
!r.slope.aspect elevation=el_rcont aspect=asp_rcont

In [ ]:
# Create Map instance
asp_rcont_map = gj.Map(filename="output/asp_rcont.png")
# Add the elevation raster to the map
asp_rcont_map.d_rast(map="el_rcont")
# Add the aspect raster to the map
asp_rcont_map.d_rast(map="asp_rcont")
asp_rcont_map.d_vect(map="elev_lid792_cont1m", col="white")
# Display map
asp_rcont_map.show()

#### Optional: create TIN model

Convert z-value stored as attribute "value" to z-coordinate.

Compute TIN:

In [ ]:
%%bash

v.to.3d elev_lid792_randpts output=elev_lid792_randpts3d column=value
v.delaunay elev_lid792_randpts3d output=elev_rand_tin
r.mapcalc "level90 = 90"

Visualize the TIN as 3D vector data:
    
Keep only "level90" and "elev_rand_tin" switched on (remove or uncheck everything else).

Switch the view from 2D to 3D. Go to Data > Vector and unckeck Show vector points. In Vector lines, change color from black to gray and set Display from on surface to as 3D. 

#### Optional: Use Python to create the data for IDW comparison

In [ ]:
for npoints in [1, 20]:
    name = 'elev_idw_1m_npoints_{}'.format(npoints)
    stats = gs.parse_command('v.surf.idw', input='elev_lid792_randpts',
                             output=name, column='value', npoints=npoints)

Computing statistics but showing only some for different number of points (you can combine the code with the code above): 

In [ ]:
for npoints in [1, 20]:
    name = 'elev_idw_1m_npoints_{}'.format(npoints)
    print("\n\n")
    print(name)
    print(len(name) * "=")
    stats = gs.parse_command('r.univar', map=name, flags='eg')

    print(stats['min'], stats['max'])

 Setting the color table and computing shaded relief for changing power (you need to create the maps before that): 

In [ ]:
for power in [0.5, 1, 2, 5]:
    name = 'elev_idw_1m_power_{}'.format(power)
    stats = gs.parse_command('v.surf.idw', input='elev_lid792_randpts',
                             output=name, column='value', power=power)
    gs.run_command('r.colors',
        map=name,
        color='elevation')
    gs.run_command('r.relief',
        input=name,
        output='elev_idw_1m_power_{}_relief'.format(power))
    gs.run_command('r.shade',
        color=name,
        shade='elev_idw_1m_power_{}_relief'.format(power),
        output='elev_idw_1m_power_{}_shaded'.format(power))

Creating a PNG image with histogram for changing power: 

In [ ]:
for power in [0.5, 1, 2, 5]:
    name = 'elev_idw_1m_power_{}'.format(power)
    power_hist = gj.Map(filename=f"output/hist_{name}.png")
    power_hist.d_histogram(map=name)
    power_hist.show()

Here are two commands often used when using the scripts. First is setting the computational. We can do that in a script, but it better and more general to do it before executing the script: 

In [ ]:
%%bash
g.region region=rural_1m

When we want to run the script again, we first need to remove the created raster maps: 

In [ ]:
%%bash
g.remove type=raster pattern="elev_idw_1m_npoints_*"

In case you don't know anything about Python scripting but you still want to try something this might be a good start together with some (free) courses at Codecademy. To learn more about using Python in GRASS GIS, see the introduction to the grass.script package. 

## Spatial interpolation and approximation II: splines (4B)

[http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/interpolation_2.html](http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/interpolation_2.html)

In [ ]:
!grass -c -e ~/grassdata/nc_spm_08_grass7/HW_interpolation_2

In [ ]:
session = gj.init("~/grassdata", "nc_spm_08_grass7", "HW_interpolation_2")

Download all text files with color rules (see above) to the selected directory. Now you can use the commands from the assignment requiring the text file without the need to specify the full path to the file. 

In [ ]:
%%bash
curl -o "inputs/deviations_color.txt" 'http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/deviations_color.txt'
curl -o "inputs/precip_color.txt" 'http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/precip_color.txt'

### Interpolate elevation raster from points using splines with different tension

 Compute aspect simultaneously with interpolation and **evaluate impact of tension by using tension=40 (default), tension=10 and tension=160.** 

In [ ]:
%%bash
g.region rural_1m res=1 -p
v.surf.rst input=elev_lid792_randpts elevation=elev_rstdef_1m aspect=asp_rstdef_1m zcolumn=value npmin=140
# v.surf.rst input=elev_lid792_randpts elevation=elev_rstdef_1m zcolumn=value aspect=asp_rstdef_1m segmax=30 npmin=140
# v.surf.rst input=elev_lid792_randpts elevation=elev_rstt10_1m aspect=asp_rstt10_1m zcolumn=value tension=10 segmax=30 npmin=140
# v.surf.rst input=elev_lid792_randpts elevation=elev_rstt160_1m aspect=asp_rstt160_1m zcolumn=value tension=160 segmax=30 npmin=140

- Compare the interpolated elevation surfaces using aspect maps.
- Change the aspect color table to grey aspect.
- Save images for your report. 

In [ ]:
%%bash
r.colors asp_rstdef_1m color=aspect
r.colors asp_rstt10_1m color=aspect
r.colors asp_rstt160_1m color=aspect

Display Tension 40

In [ ]:
# Create Map instance
asp_rst_t40_map = gj.Map(filename="output/asp_rst_t40.png")
asp_rst_t40_map.d_rast(map="elev_rstdef_1m")
asp_rst_t40_map.d_rast(map="asp_rstdef_1m")
# Display map
asp_rst_t40_map.show()

Display Tension 10

In [ ]:
# Create Map instance
asp_rst_t10_map = gj.Map(filename="output/asp_rst_t10.png")
asp_rst_t10_map.d_rast(map="asp_rstt10_1m")
# Display map
asp_rst_t10_map.show()

Display Tension 160

In [ ]:
# Create Map instance
asp_rst_t160_map = gj.Map(filename="output/asp_rst_t160.png")
asp_rst_t160_map.d_rast(map="asp_rstt160_1m")
# Display map
asp_rst_t160_map.show()

>  Or use 3D views of elev_rstdef_1m, elev_rstt10_1m, elev_rstt160_1m, make sure you switch off the aspect rasters and save the 3 images for your report. 

### Compute elevation raster and deviations vector point map

- For different values of smoothing compare deviation stats for smoothing 0.1 and 10.
- Find root mean square deviation rmse. 

In [ ]:
%%bash
v.surf.rst input=elev_lid792_randpts elevation=elev_rstdef_1mb zcolumn=value smooth=0.1 deviations=elev_rstdef_devi segmax=30 npmin=140
v.build elev_rstdef_devi
v.surf.rst input=elev_lid792_randpts elevation=elev_rstsm10_1mb zcolumn=value smooth=10 deviations=elev_rstsm10_devi segmax=30 npmin=140
v.build elev_rstsm10_devi
v.info -c elev_rstdef_devi
v.univar elev_rstdef_devi column=flt1 type=point
r.info elev_rstdef_1mb
v.info -c elev_rstsm10_devi
v.univar elev_rstsm10_devi column=flt1 type=point
r.info elev_rstsm10_1mb

Compute and display deviations maps using same color table. You need to use custom color table to see the results well.

Note that we are interpolating here the deviations, not the given elevations. 

In [ ]:
%%bash
v.surf.rst input=elev_rstdef_devi elevation=elev_rstdef_devi zcolumn=flt1 segmax=30 npmin=140
v.surf.rst input=elev_rstsm10_devi elevation=elev_rstsm10_devi zcolumn=flt1 segmax=30 npmin=140

Apply the downloaded color table deviations_color.txt to the deviation raster.

Optionally, to view the results in 3D use "elev_rstdef_1mb" for elevation (switch off everything else) and drape the deviations maps as color. 

In [ ]:
%%bash
r.colors elev_rstsm10_devi rules=inputs/deviations_color.txt
r.colors elev_rstdef_devi raster=elev_rstsm10_devi

In [ ]:
# Create Map instance
elev_rstsm10_devi_map = gj.Map(filename="output/elev_rstdef_devi.png")
elev_rstsm10_devi_map.d_rast(map="elev_rstdef_devi")
# Set legend
elev_rstsm10_devi_map.d_legend(raster="elev_rstdef_devi", at=[2,50,2,6])
# Display map
elev_rstsm10_devi_map.show()

In [ ]:
# Create Map instance
elev_rstsm10_devi_map = gj.Map(filename="output/elev_rstsm10_devi.png")
elev_rstsm10_devi_map.d_rast(map="elev_rstdef_devi")
elev_rstsm10_devi_map.d_rast(map="elev_rstsm10_devi")
# Set legend
elev_rstsm10_devi_map.d_legend(raster="elev_rstsm10_devi", at=[2,50,2,6])
# Display map
elev_rstsm10_devi_map.show()

### Compute predictive error of interpolation

Compute predictive error of interpolation for each point using cross-validation (no raster output, only points with pred. errors). 

In [ ]:
%%bash
v.surf.rst -c input=elev_lid792_randpts zcolumn=value cvdev=elev_rstdef_cv npmin=120 segmax=35
v.build elev_rstdef_cv
v.univar elev_rstdef_cv column=flt1 type=point

Compute raster map of predictive errors and **identify locations where the sampling is inadequate.**

Optionally, to view the result in 3D use "elev_rstdef_1mb" for elevation (switch off everything else) and drape the crossvalidation map "elev_rstdef_cv" as color. 

In [ ]:
%%bash
v.surf.rst input=elev_rstdef_cv elevation=elev_rstdef_cv zcolumn=flt1
r.colors elev_rstdef_cv raster=elev_rstsm10_devi

In [ ]:
# Create Map instance
elev_rstdef_cv_map = gj.Map(filename="output/elev_rstdef_cv.png")
elev_rstdef_cv_map.d_rast(map="elev_rstdef_cv")
elev_rstdef_cv_map.d_vect(map="elev_rstdef_cv", size=2)
# Set legend
elev_rstdef_cv_map.d_legend(raster="elev_rstdef_cv", at=[2,50,2,6])
# Display map
elev_rstdef_cv_map.show()

### Interpolate precipitation with influence of topography

- Set the 3D region (read the man page for [g.region](https://grass.osgeo.org/grass76/manuals/g.region.html)).
- We set tbres to high value 
    > we have just a single level because we are not computing the 3D raster (see lecture for more details). 

In [ ]:
%%bash
g.region raster=elev_state_500m -p
g.region t=2000 b=0 tbres=2000 res3=500 -p3

Compute precipitation raster map without influence of elevation (with segmax=700 segmentation is not performed so interpolation function is computed using all points at once).
We will use mask during the interpolation. 

In [ ]:
%%bash
r.mask raster=ncmask_500m
v.info -c precip_30ynormals
v.surf.rst input=precip_30ynormals elevation=precip_annual_500m zcolumn=annual segmax=700

Use the downloaded the color table precip_color.txt.

Zoom to computational region when displaying the result. 

In [ ]:
!r.colors precip_annual_500m rules=inputs/precip_color.txt

In [ ]:
# Create Map instance
precip_annual_500m_map = gj.Map(filename="output/precip_annual_500m.png")
precip_annual_500m_map.d_rast(map="precip_annual_500m")
# Set legend
precip_annual_500m_map.d_legend(raster="precip_annual_500m", at=[2,50,2,6])
# Display map
precip_annual_500m_map.show()

Compute precipitation raster map with elevation. 

There is both 3D voxel output and 2D raster output - we want the 2D raster output (cross_output).

Optionally to view the results in 3D, switch off everything except for elev_state_500m and precip_30ynormals_3d,
switch to 3D, set(type in) viewer height at 300000, z-exag at 6, fine res=1,
use precip_anntopo_500m for color, set icon size for points - sphere, 5000.
Display the result and save the image for the report. 

In [ ]:
%%bash
v.info -c precip_30ynormals_3d
v.vol.rst input=precip_30ynormals_3d cross_input=elev_state_500m cross_output=precip_anntopo_500m maskmap=elev_state_500m wcolumn=annual zscale=90 segmax=700
r.colors precip_anntopo_500m raster=precip_annual_500m

In [ ]:
# Create Map instance
precip_anntopo_500m_map = gj.Map(filename="output/precip_anntopo_500m.png")
precip_anntopo_500m_map.d_rast(map="precip_anntopo_500m")
# Set legend
precip_anntopo_500m_map.d_legend(raster="precip_anntopo_500m", at=[2,50,2,6])
# Display map
precip_anntopo_500m_map.show()

**Try to explain how was elevation used for the precipitation raster interpolation.** 

After you are finished, remove mask. 

In [ ]:
!r.mask -r